## Pysgrid Interpolation Demo 
 
Lets start with the pysgrid object.

In [1]:
from netCDF4 import Dataset
import pysgrid
import numpy as np

url = ('http://geoport.whoi.edu/thredds/dodsC/clay/usgs/users/'
               'jcwarner/Projects/Sandy/triple_nest/00_dir_NYB05.ncml')

sgrid = pysgrid.load_grid(Dataset(url))


Pysgrid variable objects can now serve data, similar to pyugrid variables. It calls directly on the netCDF Variable object, and has it's lazy-loading behavior.

In [2]:
print sgrid.temp
print sgrid.temp.shape

SGridVariable object: sea_water_potential_temperature, on the faces
(201, 16, 107, 345)


In [3]:
sgrid.temp[0,0,50:55,100:105]

array([[ 18.07930946,  18.14973068,  18.2201519 ,  18.21634102,
         18.21252823],
       [ 18.15918732,  18.22960854,  18.30002975,  18.29621696,
         18.29240417],
       [ 18.23906708,  18.30948639,  18.37990761,  18.37609482,
         18.32413864],
       [ 18.31894493,  18.38936615,  18.45978546,  18.40782738,
         18.35586929],
       [ 18.45632744,  18.52674675,  18.50500488,  18.4530468 ,
         18.40108871]], dtype=float32)

First lets establish some points on the grid we're interested in. For simplicity, we'll derive them from the grid nodes.

In [4]:
points_lon = sgrid.node_lon[50:52,100:105] * 0.9999
points_lat = sgrid.node_lat[50:52,100:105] * 0.9999
points = np.stack((points_lon, points_lat), axis = -1).reshape(-1,2)
print points

[[-73.74568073  40.32299155]
 [-73.7391676   40.32583585]
 [-73.73265448  40.32868015]
 [-73.72614136  40.33152445]
 [-73.71962823  40.33436874]
 [-73.74901747  40.33079974]
 [-73.7425043   40.33364399]
 [-73.73599114  40.33648823]
 [-73.72947797  40.33933248]
 [-73.72296481  40.34217672]]


The SGrid object now provides a way to get the x,y indices of these points on the node grid. 

In [5]:
node_ind = sgrid.locate_faces(points)
print node_ind

[[ 49 100]
 [ 49 101]
 [ 49 102]
 [ 49 103]
 [ 49 104]
 [ 50 100]
 [ 50 101]
 [ 50 102]
 [ 50 103]
 [ 50 104]]


### Scenario: I want to interpolate the temperature and salinity at a list of points.

In [13]:
%%time
interp_temps = sgrid.interpolate_var_to_points(points, sgrid.temp[0,6])
interp_salinity = sgrid.interpolate_var_to_points(points, sgrid.salt[0,6])

print interp_temps
print interp_salinity

[23.40559527721179 23.391546600118488 23.41423669831296 23.436921401441918
 23.45776700834991 23.419357486410757 23.4053081701454 23.427996148979414
 23.44884449221559 23.45918945470487]
[43.41301646604848 43.40833575274081 43.4036551192405 43.39897456556031
 43.394294092083555 43.41151307230791 43.40683296250011 43.4021529311478
 43.39747298010166 43.39279310876989]
Wall time: 18 ms


Done!

Interpolating for a vector quantity like velocity is a bit more complicated, but the same approach can be used piecewise.

In [11]:
%%time
interp_u = sgrid.interpolate_var_to_points(points, sgrid.u[0,6])
interp_v = sgrid.interpolate_var_to_points(points, sgrid.v[0,6])
print np.column_stack((interp_u, interp_v))

[[ 0.02588658 -0.19636827]
 [ 0.02969118 -0.19708817]
 [ 0.03296548 -0.19143968]
 [ 0.03621145 -0.18579166]
 [ 0.03945695 -0.1801441 ]
 [ 0.03022276 -0.19490993]
 [ 0.03349701 -0.19477434]
 [ 0.03674293 -0.18912618]
 [ 0.03998839 -0.18347849]
 [ 0.04323336 -0.17783126]]
Wall time: 22 ms


Trying to interpolate to points outside the grid will return nans.

In [8]:
bad_pts = np.array(([0,0],[1,1]))
bad_ind = sgrid.locate_faces(bad_pts)
print bad_ind

[[-1 -1]
 [-1 -1]]


In [9]:
interp_u = sgrid.interpolate_var_to_points(bad_pts, sgrid.u[0,6])
print interp_u

[nan nan]


If you are interpolating multiple variables at the same time or interpolating to the same points repeatedly, you can save considerable computation by pre-computing some information for interpolate_var_to_points.

In [12]:
%%time
indices = sgrid.locate_faces(points)

#re-use these variables if you plan to interpolate to the same points repeatedly
center_inds = sgrid.translate_index(points, indices, 'center')
interp_alphas = sgrid.interpolation_alphas(points, center_inds, 'center')

interp_temps = sgrid.interpolate_var_to_points(points, sgrid.temp[0,6], center_inds, interp_alphas)
interp_salinity = sgrid.interpolate_var_to_points(points, sgrid.salt[0,6], center_inds, interp_alphas)

print interp_temps
print interp_salinity

[23.40559527721179 23.391546600118488 23.41423669831296 23.436921401441918
 23.45776700834991 23.419357486410757 23.4053081701454 23.427996148979414
 23.44884449221559 23.45918945470487]
[43.41301646604848 43.40833575274081 43.4036551192405 43.39897456556031
 43.394294092083555 43.41151307230791 43.40683296250011 43.4021529311478
 43.39747298010166 43.39279310876989]
Wall time: 13 ms
